# Predicción de Retrasos de Vuelos ✈️
Este notebook entrena modelos (XGBoost y CatBoost) para predecir retrasos graves en vuelos, evalúa su desempeño y guarda los modelos en la carpeta `ds/model/`.

In [ ]:
# Importar librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from joblib import dump, load

In [ ]:
# Cargar dataset limpio
datos = pd.read_csv("ds/datos/flight_clean.csv")
datos.head()

In [ ]:
# Función de entrenamiento de modelos
def entrenar_modelos(datos):
    # Preparación de datos
    X = datos[['RETRASO_AVIÓN_TARDÍO','RETRASO_AEROLÍNEA',
               'RETRASO_SISTEMA_AÉREO','RETRASO_CLIMA']]
    y = datos['RETRASO_GRAVE']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Entrenamiento XGBoost
    xgb = XGBClassifier(scale_pos_weight=8.1,
                        eval_metric="logloss",
                        use_label_encoder=False,
                        random_state=42)
    xgb.fit(X_train, y_train)

    # Entrenamiento CatBoost
    cat = CatBoostClassifier(iterations=500,
                             learning_rate=0.1,
                             depth=6,
                             loss_function='Logloss',
                             random_seed=42,
                             verbose=0)
    cat.fit(X_train, y_train)

    # Ensemble
    proba_xgb = xgb.predict_proba(X_test)[:,1]
    proba_cat = cat.predict_proba(X_test)[:,1]
    proba_ensemble = (proba_xgb + proba_cat) / 2

    # Evaluación
    y_pred_final = (proba_ensemble >= 0.6).astype(int)
    cm = confusion_matrix(y_test, y_pred_final)

    print("Reporte final con umbral 0.6:")
    print(classification_report(y_test, y_pred_final))

    # Visualización
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicción")
    plt.ylabel("Real")
    plt.show()

    # Guardar modelos
    dump(xgb, "ds/model/xgb_model.joblib")
    dump(cat, "ds/model/cat_model.joblib")

    return xgb, cat

In [ ]:
# Entrenar y guardar modelos
xgb, cat = entrenar_modelos(datos)

In [ ]:
# Función para cargar modelos guardados
def cargar_modelo(ruta):
    return load(ruta)

# Función para predecir con nuevos datos
def predecir_retraso(modelo, nuevo_dato):
    return modelo.predict(nuevo_dato)